In [1]:
import sys
principal_path = '../'
if principal_path not in sys.path:
    sys.path.append('../')

In [2]:
import pandas as pd
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torch.utils.data.dataset import Subset

from models.PlasticityMNISTModel import PlasticityMNISTModel
from models.MNISTModel import MNISTModel
from utils import load_data_filtered, find_low_activation_neurons

In [3]:
SEED = 2024
pl.seed_everything(SEED)

Seed set to 2024


2024

In [4]:
checkpoint_ffn = 'checkpoints/mnist_model.ckpt'
model = MNISTModel.load_from_checkpoint(checkpoint_ffn)
model.eval()
print(model)

MNISTModel(
  (train_acc): MulticlassAccuracy()
  (val_acc): MulticlassAccuracy()
  (test_acc): MulticlassAccuracy()
  (precision): MulticlassPrecision()
  (recall): MulticlassRecall()
  (f1): MulticlassF1Score()
  (layer_1): Linear(in_features=784, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=10, bias=True)
)


# [0,1]

In [5]:
classes_to_include = [0, 1]
BATCH_SIZE = 32
train_loader_filtered, val_loader_filtered, test_loader_filtered = load_data_filtered(BATCH_SIZE, classes_to_include, num_workers=0)

In [6]:
mnist_model_trainer = pl.Trainer(max_epochs=5)
# test model
mnist_model_trainer.test(model, test_loader_filtered)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\code\ExAI\exai\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html

Testing DataLoader 0: 100%|██████████| 67/67 [00:00<00:00, 68.62it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           f1               0.8955720067024231
        precision           0.8991332650184631
         recall             0.8922649025917053
        test_acc            0.9900709390640259
        test_loss          0.050506915897130966
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.050506915897130966,
  'test_acc': 0.9900709390640259,
  'precision': 0.8991332650184631,
  'recall': 0.8922649025917053,
  'f1': 0.8955720067024231}]

In [7]:
df = pd.read_csv("activations/activations_minist_model.csv")
df.head()

,Number,Layer1_Neuron1,Layer1_Neuron2,Layer1_Neuron3,Layer1_Neuron4,Layer1_Neuron5,Layer1_Neuron6,Layer1_Neuron7,Layer1_Neuron8,Layer1_Neuron9,...,Layer3_Neuron1,Layer3_Neuron2,Layer3_Neuron3,Layer3_Neuron4,Layer3_Neuron5,Layer3_Neuron6,Layer3_Neuron7,Layer3_Neuron8,Layer3_Neuron9,Layer3_Neuron10
0,7.0,0.000000,0.000000,0.000000,0.399097,0.000000,0.000000,0.000000,0.000000,0.000000,...,-22.055817,-17.074514,-12.136678,-11.195572,-25.313034,-18.126488,-34.183750,-0.000019,-16.948915,-15.565269
1,2.0,0.000000,0.000000,3.122375,2.949144,2.404661,1.715258,0.338522,0.128126,1.610624,...,-15.903490,-8.095853,-0.000322,-11.127125,-27.803934,-17.198702,-14.273718,-19.698458,-13.450046,-30.231627
2,1.0,1.827291,1.748518,0.699948,0.539016,0.000000,0.628443,0.588888,0.000000,0.020213,...,-16.210146,-0.000385,-11.081397,-12.736809,-10.941203,-11.521659,-11.719425,-8.667583,-8.749459,-14.620137
3,0.0,0.000000,0.000000,0.614674,0.000000,1.043494,0.438858,0.000000,3.829711,2.498251,...,-0.000426,-13.427382,-9.176719,-14.582230,-15.579443,-13.185836,-8.165977,-12.736848,-13.246180,-10.434205
4,4.0,0.000000,0.000000,1.187802,0.092359,0.000000,0.000000,0.000000,2.307845,1.650951,...,-13.418967,-15.270522,-12.993325,-18.622927,-0.005147,-10.958460,-15.022395,-10.377352,-9.788157,-5.293344


In [8]:
def get_layer_columns(layer):
    layer_columns = ['Number']
    layer_columns += [c for c in df.columns if layer in c]
    return layer_columns

df_layer1 = df[get_layer_columns('Layer1')]
df_layer2 = df[get_layer_columns('Layer2')]
#df_layer3 = df[get_layer_columns('Layer3')]

In [9]:
threshold_layer_1 = 0.5
print(f'Threshold layer 1: {threshold_layer_1*100}%')

mask_indices_layer_1 = find_low_activation_neurons(df_layer1, 0, 1, threshold=threshold_layer_1)
print(f'High activation neurons layer: {128 - len(mask_indices_layer_1)}')

threshold_layer_2 = 0.5
print(f'Threshold layer 2: {threshold_layer_2*100}%')

mask_indices_layer_2 = find_low_activation_neurons(df_layer2, 0, 1, threshold=threshold_layer_2)

print(f'High activation neurons layer: {256 - len(mask_indices_layer_2)}')

Threshold layer 1: 50.0%
High activation neurons layer: 25
Threshold layer 2: 50.0%
High activation neurons layer: 56


In [10]:
# load plasticy model
plasticy_model = PlasticityMNISTModel.load_from_checkpoint(checkpoint_ffn, mask_indices_layer_1=mask_indices_layer_1, mask_indices_layer_2=mask_indices_layer_2)
print(plasticy_model)

PlasticityMNISTModel(
  (train_acc): MulticlassAccuracy()
  (val_acc): MulticlassAccuracy()
  (test_acc): MulticlassAccuracy()
  (precision): MulticlassPrecision()
  (recall): MulticlassRecall()
  (f1): MulticlassF1Score()
  (layer_1): Linear(in_features=784, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=10, bias=True)
)


In [11]:
plasticy_model_trainer = pl.Trainer(max_epochs=5)
plasticy_model_trainer.fit(plasticy_model, train_loader_filtered, val_loader_filtered)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params
--------------------------------------------------
0 | train_acc | MulticlassAccuracy  | 0     
1 | val_acc   | MulticlassAccuracy  | 0     
2 | test_acc  | MulticlassAccuracy  | 0     
3 | precision | MulticlassPrecision | 0     
4 | recall    | MulticlassRecall    | 0     
5 | f1        | MulticlassF1Score   | 0     
6 | layer_1   | Linear              | 100 K 
7 | layer_2   | Linear              | 33.0 K
8 | layer_3   | Linear              | 2.6 K 
--------------------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


c:\code\ExAI\exai\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
c:\code\ExAI\exai\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]Failed to save model graph: 'ExperimentWriter' object has no attribute 'add_graph'
Epoch 4: 100%|██████████| 317/317 [00:02<00:00, 147.48it/s, v_num=8, train_loss=0.000, train_acc=1.000, val_loss=8.57e-6, val_acc=1.000]  

`Trainer.fit` stopped: `max_epochs=5` reached.


Failed to save weights histogram: 'ExperimentWriter' object has no attribute 'add_histogram'
Epoch 4: 100%|██████████| 317/317 [00:02<00:00, 147.07it/s, v_num=8, train_loss=0.000, train_acc=1.000, val_loss=8.57e-6, val_acc=1.000]


In [12]:
plasticy_model_trainer.test(plasticy_model, test_loader_filtered)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\code\ExAI\exai\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 67/67 [00:00<00:00, 143.15it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           f1               0.9995154142379761
        precision           0.9996017813682556
         recall             0.9994596838951111
        test_acc            0.9995272159576416
        test_loss          0.0035234333481639624
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.0035234333481639624,
  'test_acc': 0.9995272159576416,
  'precision': 0.9996017813682556,
  'recall': 0.9994596838951111,
  'f1': 0.9995154142379761}]

In [13]:
# save checkpoint
plasticy_model_trainer.save_checkpoint('checkpoints/plasticy_mnist_model.ckpt')

### study weigths

In [15]:
# load plasticy model
plasticy_model = PlasticityMNISTModel.load_from_checkpoint('checkpoints/plasticy_mnist_model.ckpt', mask_indices_layer_1=mask_indices_layer_1, mask_indices_layer_2=mask_indices_layer_2)
plasticy_model.eval()
print(plasticy_model)


PlasticityMNISTModel(
  (train_acc): MulticlassAccuracy()
  (val_acc): MulticlassAccuracy()
  (test_acc): MulticlassAccuracy()
  (precision): MulticlassPrecision()
  (recall): MulticlassRecall()
  (f1): MulticlassF1Score()
  (layer_1): Linear(in_features=784, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=10, bias=True)
)


In [17]:
plasticy_model.layer_1.weight.data

tensor([[-0.0270, -0.0021,  0.0056,  ...,  0.0151,  0.0341,  0.0220],
        [ 0.0315,  0.0085, -0.0135,  ...,  0.0088,  0.0255,  0.0351],
        [-0.0356,  0.0065, -0.0031,  ...,  0.0211,  0.0130, -0.0242],
        ...,
        [ 0.0276,  0.0313,  0.0158,  ...,  0.0118,  0.0186,  0.0071],
        [ 0.0226,  0.0201,  0.0059,  ...,  0.0301,  0.0083,  0.0342],
        [ 0.0328, -0.0055, -0.0058,  ...,  0.0305,  0.0073,  0.0319]])

In [16]:
#load minist model
mnist_model = MNISTModel.load_from_checkpoint(checkpoint_ffn)
mnist_model.eval()
print(mnist_model)

MNISTModel(
  (train_acc): MulticlassAccuracy()
  (val_acc): MulticlassAccuracy()
  (test_acc): MulticlassAccuracy()
  (precision): MulticlassPrecision()
  (recall): MulticlassRecall()
  (f1): MulticlassF1Score()
  (layer_1): Linear(in_features=784, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=10, bias=True)
)
